# Spatial and Longitudinal Analysis in Active Transportation Research

This notebook demonstrates how to use the spatial and longitudinal data handling capabilities of the CISD package for active transportation research.

## Overview

Transportation infrastructure interventions often have:
1. **Spatial spillover effects**: The impact can extend beyond the immediate intervention area
2. **Temporal dynamics**: Effects can evolve over time after implementation

This tutorial covers:
1. Working with geospatial data in causal inference
2. Handling longitudinal (panel) data in before-after studies
3. Accounting for spatial dependencies in treatment effects
4. Event study analysis of intervention timing

## Setup and Dependencies

First, let's import the necessary libraries and configure the environment.

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Import CISD modules
import sys
sys.path.append('..')

# Spatial dependencies
try:
    import geopandas as gpd
    from shapely.geometry import Point
    import contextily as ctx
    HAS_SPATIAL = True
except ImportError:
    HAS_SPATIAL = False
    print("Spatial dependencies require: pip install geopandas shapely contextily libpysal")

# Import CISD spatial-temporal utilities
try:
    from cisd.spatial_temporal import (SpatialDependencyHandler,
                                     LongitudinalDataHandler, 
                                     create_spatial_panel_data)
    from cisd.visualization import plot_spatial_effects
except ImportError as e:
    print(f"Error importing spatial dependencies: {e}")

# Configure plotting
plt.style.use('seaborn-whitegrid')
sns.set_context("talk")
%matplotlib inline

## 1. Creating a Synthetic Spatial Dataset

Let's create a synthetic dataset with spatial properties based on a real geographic area.

In [ ]:
def create_synthetic_spatial_data(n_points=100, bbox=None, random_state=42):
    """
    Create a synthetic point dataset with spatial properties.
    
    Parameters
    ----------
    n_points : int, default=100
        Number of points to generate
    bbox : tuple, optional
        Bounding box (xmin, ymin, xmax, ymax)
    random_state : int, default=42
        Random seed
        
    Returns
    -------
    gdf : geopandas.GeoDataFrame
        GeoDataFrame with synthetic points
    """
    if not HAS_SPATIAL:
        raise ImportError("This function requires geopandas and shapely.")
    
    np.random.seed(random_state)
    
    # Default to Salt Lake City area if no bbox provided
    if bbox is None:
        bbox = (-112.1, 40.7, -111.8, 40.85)  # Salt Lake City approximate bbox
    
    # Generate random points within the bbox
    x = np.random.uniform(bbox[0], bbox[2], n_points)
    y = np.random.uniform(bbox[1], bbox[3], n_points)
    
    # Create geometry column
    geometry = [Point(xy) for xy in zip(x, y)]
    
    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(
        {
            'location_id': range(n_points),
            'walkability': np.random.normal(5, 1.5, n_points),
            'population': np.random.poisson(1000, n_points),
            'income': np.random.lognormal(10, 0.5, n_points)
        },
        geometry=geometry,
        crs="EPSG:4326"  # WGS84 coordinate system
    )
    
    # Create treatment variable (more likely near the center)
    center_x = (bbox[0] + bbox[2]) / 2
    center_y = (bbox[1] + bbox[3]) / 2
    
    # Calculate distance to center
    gdf['dist_to_center'] = gdf.geometry.apply(
        lambda p: ((p.x - center_x)**2 + (p.y - center_y)**2)**0.5
    )
    
    # Treatment more likely close to center
    p_treatment = 1 / (1 + np.exp(5 * gdf['dist_to_center'] / gdf['dist_to_center'].max()))
    gdf['treatment'] = np.random.binomial(1, p_treatment)
    
    # Generate outcome with spatial correlation
    # Base outcome depends on covariates
    outcome_base = 0.5 * gdf['walkability'] + 0.3 * np.log(gdf['income'] / 10000)
    
    # Treatment effect
    treatment_effect = 2.0 * gdf['treatment']
    
    # Add spatially correlated noise
    # Simple approach: closer points have more similar noise
    from sklearn.metrics.pairwise import euclidean_distances
    coords = np.column_stack([gdf.geometry.x, gdf.geometry.y])
    dist_matrix = euclidean_distances(coords)
    
    # Convert distances to similarities
    sim_matrix = np.exp(-dist_matrix / dist_matrix.mean())
    
    # Generate correlated noise
    noise_seed = np.random.normal(0, 1, n_points)
    spatial_noise = sim_matrix @ noise_seed / sim_matrix.sum(axis=1)
    
    # Final outcome
    gdf['outcome'] = outcome_base + treatment_effect + spatial_noise + np.random.normal(0, 0.5, n_points)
    
    return gdf

# Create synthetic spatial data
if HAS_SPATIAL:
    spatial_df = create_synthetic_spatial_data(n_points=150)
    print(f"Created synthetic spatial dataset with {len(spatial_df)} points")
    spatial_df.head()
else:
    print("Skipping spatial data creation due to missing dependencies.")

## 2. Visualizing the Spatial Data

Let's plot our synthetic data to understand its spatial structure.

In [ ]:
if HAS_SPATIAL:
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Plot 1: Treatment assignment
    spatial_df.plot(
        column='treatment', 
        ax=axes[0],
        categorical=True,
        legend=True,
        markersize=50,
        alpha=0.7,
        cmap='viridis'
    )
    axes[0].set_title('Treatment Assignment')
    
    # Plot 2: Walkability
    spatial_df.plot(
        column='walkability',
        ax=axes[1],
        legend=True,
        markersize=50,
        alpha=0.7,
        cmap='YlOrRd'
    )
    axes[1].set_title('Walkability Score')
    
    # Plot 3: Outcome
    spatial_df.plot(
        column='outcome',
        ax=axes[2],
        legend=True,
        markersize=50,
        alpha=0.7,
        cmap='RdYlGn'
    )
    axes[2].set_title('Outcome Variable')
    
    # Add basemaps if contextily is available
    try:
        for ax in axes:
            ctx.add_basemap(
                ax, 
                crs=spatial_df.crs.to_string(),
                source=ctx.providers.CartoDB.Positron
            )
            ax.set_axis_off()
    except Exception as e:
        print(f"Error adding basemap: {e}")
    
    plt.tight_layout()
else:
    print("Skipping visualization due to missing dependencies.")

## 3. Handling Spatial Dependencies in Causal Inference

Now we'll use the `SpatialDependencyHandler` class to account for spatial autocorrelation in our analysis.

In [ ]:
if HAS_SPATIAL:
    # Initialize spatial dependency handler
    sdh = SpatialDependencyHandler(weight_type='queen', standardize=True)
    
    # Fit the handler to our spatial data
    sdh.fit(spatial_df)
    
    # Create spatially lagged variables
    spatial_df_with_lags = sdh.transform(spatial_df)
    
    # Print new columns
    lag_columns = [col for col in spatial_df_with_lags.columns if 'spatial_lag' in col]
    print(f"Created {len(lag_columns)} spatial lag variables:")
    for col in lag_columns:
        print(f"  - {col}")
    
    # Examine correlation between variables and their spatial lags
    for col in ['walkability', 'income', 'outcome']:
        lag_col = f"{col}_spatial_lag"
        if lag_col in spatial_df_with_lags.columns:
            corr = spatial_df_with_lags[[col, lag_col]].corr().iloc[0, 1]
            print(f"Correlation between {col} and its spatial lag: {corr:.3f}")
    
    spatial_df_with_lags.head()
else:
    print("Skipping spatial dependency analysis due to missing dependencies.")

In [ ]:
if HAS_SPATIAL:
    # Compare naive vs. spatially adjusted treatment effects
    from sklearn.linear_model import LinearRegression
    
    # Naive estimate (ignoring spatial dependence)
    X_naive = spatial_df[['walkability', 'income']]
    D = spatial_df['treatment']
    Y = spatial_df['outcome']
    
    # Add treatment to features
    X_naive_with_D = pd.concat([X_naive, D], axis=1)
    
    # Fit naive model
    naive_model = LinearRegression()
    naive_model.fit(X_naive_with_D, Y)
    naive_effect = naive_model.coef_[-1]  # Last coefficient is treatment effect
    
    # Spatially-adjusted estimate
    X_spatial = spatial_df_with_lags[['walkability', 'income', 
                                    'walkability_spatial_lag', 
                                    'income_spatial_lag',
                                    'outcome_spatial_lag']]
    
    # Add treatment to features
    X_spatial_with_D = pd.concat([X_spatial, D], axis=1)
    
    # Fit spatial model
    spatial_model = LinearRegression()
    spatial_model.fit(X_spatial_with_D, Y)
    spatial_effect = spatial_model.coef_[-1]  # Last coefficient is treatment effect
    
    print(f"Treatment effect (naive): {naive_effect:.3f}")
    print(f"Treatment effect (spatially adjusted): {spatial_effect:.3f}")
    print(f"Difference: {spatial_effect - naive_effect:.3f}")
    
    # Other approach: use spatial dependency handler directly
    # Calculate individual treatment effects (Y - counterfactual Y)
    individual_effects = (Y - naive_model.predict(X_naive_with_D)).values
    
    # Adjust for spatial autocorrelation
    adjusted_effects = sdh.adjust_effect_estimates(individual_effects, spatial_df)
    
    # Compare mean effects
    mean_individual_effect = individual_effects.mean()
    mean_adjusted_effect = adjusted_effects.mean()
    
    print(f"\nMean individual effect: {mean_individual_effect:.3f}")
    print(f"Mean adjusted effect: {mean_adjusted_effect:.3f}")
else:
    print("Skipping spatial treatment effect analysis due to missing dependencies.")

## 4. Visualizing Spatial Treatment Effects

Let's visualize the spatial distribution of treatment effects to identify patterns.

In [ ]:
if HAS_SPATIAL:
    # Add individual treatment effects to the dataframe
    spatial_df['individual_effect'] = individual_effects
    spatial_df['adjusted_effect'] = adjusted_effects
    
    # Create plots using the visualization utility
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Plot unadjusted effects
    plot_spatial_effects(
        spatial_df, 
        'individual_effect', 
        title='Unadjusted Treatment Effects',
        figsize=(8, 8)
    )
    
    plt.tight_layout()
    plt.show()
    
    # Plot adjusted effects
    plot_spatial_effects(
        spatial_df, 
        'adjusted_effect', 
        title='Spatially Adjusted Treatment Effects',
        figsize=(8, 8)
    )
    
    plt.tight_layout()
else:
    print("Skipping spatial effect visualization due to missing dependencies.")

## 5. Creating a Synthetic Longitudinal Dataset

Now let's create a synthetic longitudinal (panel) dataset to demonstrate methods for time-varying treatments.

In [ ]:
def create_synthetic_panel_data(n_units=50, n_periods=10, treatment_time=5, random_state=42):
    """
    Create synthetic panel data for active transportation research.
    
    Parameters
    ----------
    n_units : int, default=50
        Number of units (e.g., neighborhoods)
    n_periods : int, default=10
        Number of time periods
    treatment_time : int, default=5
        Time period when treatment starts
    random_state : int, default=42
        Random seed
        
    Returns
    -------
    panel_df : pandas.DataFrame
        Panel dataset with units, time periods, and outcomes
    """
    np.random.seed(random_state)
    
    # Create unit characteristics (time-invariant)
    unit_chars = {
        'neighborhood_size': np.random.gamma(5, 2, n_units),
        'baseline_walkability': np.random.normal(5, 2, n_units),
        'income_level': np.random.lognormal(10, 0.5, n_units),
    }
    
    # Assign treatment to half the units
    treatment_assignment = np.zeros(n_units)
    treatment_assignment[:n_units//2] = 1
    np.random.shuffle(treatment_assignment)
    
    # Create panel data
    panel_data = []
    
    # Unit-specific trends
    unit_trends = np.random.normal(0.1, 0.05, n_units)  # Each unit has slightly different trend
    
    # Initial active transportation rates
    base_outcomes = 20 + 2 * unit_chars['baseline_walkability'] - 0.1 * np.log(unit_chars['income_level'])
    
    # Generate observations for each unit and time period
    for unit in range(n_units):
        for period in range(n_periods):
            # Treatment indicator (1 if treated unit and after treatment time)
            treatment = 1 if (treatment_assignment[unit] == 1 and period >= treatment_time) else 0
            
            # Outcome equation components:
            # 1. Base outcome for this unit
            outcome = base_outcomes[unit]
            
            # 2. Time trend specific to this unit
            outcome += period * unit_trends[unit]
            
            # 3. Seasonal effects (sine wave with period 4)
            outcome += 1.5 * np.sin(period * np.pi / 2)
            
            # 4. Treatment effect (growing over time after treatment)
            if treatment == 1:
                # Treatment effect grows over time after implementation
                time_since_treatment = period - treatment_time + 1
                treatment_effect = 3 * (1 - np.exp(-0.5 * time_since_treatment))
                outcome += treatment_effect
            
            # 5. Random noise
            outcome += np.random.normal(0, 1)
            
            # Add record to panel data
            panel_data.append({
                'unit_id': unit,
                'period': period,
                'treatment': treatment,
                'outcome': outcome,
                'neighborhood_size': unit_chars['neighborhood_size'][unit],
                'baseline_walkability': unit_chars['baseline_walkability'][unit],
                'income_level': unit_chars['income_level'][unit],
                # Add time-varying covariates
                'precipitation': np.random.gamma(2, 2),  # mm of rain
                'gas_price': 3 + 0.05 * period + np.random.normal(0, 0.1)  # Increasing gas prices
            })
    
    return pd.DataFrame(panel_data)

# Create synthetic panel data
panel_df = create_synthetic_panel_data(n_units=80, n_periods=12)
print(f"Created synthetic panel data with {len(panel_df)} observations")
panel_df.head()

## 6. Exploring the Panel Data

Let's explore our synthetic panel data to understand its structure.

In [ ]:
# Calculate summary statistics
panel_summary = panel_df.groupby(['period', 'treatment'])['outcome'].agg(['mean', 'std', 'count'])
print("Summary statistics by period and treatment:")
print(panel_summary)

# Plot time trends for treated and control groups
plt.figure(figsize=(12, 6))

# Calculate mean outcomes by period and treatment status
trends = panel_df.groupby(['period', 'treatment'])['outcome'].mean().unstack()

# Plot trends
plt.plot(trends.index, trends[0], 'b-', label='Control Group')
plt.plot(trends.index, trends[1], 'r-', label='Treated Group')

# Add reference line at treatment time
plt.axvline(x=5, color='g', linestyle='--', label='Treatment Start')

plt.xlabel('Time Period')
plt.ylabel('Mean Outcome')
plt.title('Time Trends in Outcome by Treatment Status')
plt.legend()
plt.grid(True, alpha=0.3)

plt.show()

## 7. Difference-in-Differences Analysis

Now let's use the `LongitudinalDataHandler` to perform difference-in-differences analysis.

In [ ]:
# Initialize longitudinal data handler for diff-in-diff
ldh_did = LongitudinalDataHandler(method='did', aggregation='mean')

# Fit the handler to our panel data
ldh_did.fit(
    X=panel_df,
    D=panel_df['treatment'],
    Y=panel_df['outcome'],
    time_var='period',
    id_var='unit_id'
)

# Get the treatment effect estimate
did_effect = ldh_did.estimate_effect()
print(f"Difference-in-Differences treatment effect estimate: {did_effect:.3f}")

## 8. Event Study Analysis

Let's create an event study plot to see how the treatment effect evolves over time.

In [ ]:
# Create event study plot
event_study_plot = ldh_did.event_study_plot(time_range=(-5, 6))
plt.show()

## 9. Fixed Effects Analysis

Now let's use a two-way fixed effects model to account for unit-specific and time-specific factors.

In [ ]:
# Initialize longitudinal data handler for fixed effects
ldh_fe = LongitudinalDataHandler(method='fe')

# Fit the handler to our panel data
ldh_fe.fit(
    X=panel_df,
    D=panel_df['treatment'],
    Y=panel_df['outcome'],
    time_var='period',
    id_var='unit_id'
)

# Get the treatment effect estimate
fe_effect = ldh_fe.estimate_effect()
print(f"Fixed Effects treatment effect estimate: {fe_effect:.3f}")

# Compare with DID estimate
print(f"Difference from DID estimate: {fe_effect - did_effect:.3f}")

## 10. Combining Spatial and Longitudinal Analysis

For a real application, we might want to combine spatial and longitudinal methods. Let's demonstrate this approach with a synthetic spatio-temporal dataset.

In [ ]:
if HAS_SPATIAL:
    try:
        # Create a spatial panel dataset
        spatial_panel_df = create_spatial_panel_data(
            geo_df=spatial_df,
            n_periods=8,
            treatment_time=4,
            spatial_correlation=0.3,
            temporal_correlation=0.7,
            seed=123
        )
        
        print(f"Created spatial panel dataset with {len(spatial_panel_df)} observations")
        print(spatial_panel_df.head())
        
        # Plot outcomes over time by treatment status
        plt.figure(figsize=(12, 6))
        
        # Calculate mean outcomes by period and treatment status
        spatio_trends = spatial_panel_df.groupby(['time', 'treatment'])['outcome'].mean().unstack()
        
        # Plot trends
        plt.plot(spatio_trends.index, spatio_trends[0], 'b-', label='Control Group')
        plt.plot(spatio_trends.index, spatio_trends[1], 'r-', label='Treated Group')
        
        # Add reference line at treatment time
        plt.axvline(x=4, color='g', linestyle='--', label='Treatment Start')
        
        plt.xlabel('Time Period')
        plt.ylabel('Mean Outcome')
        plt.title('Spatio-temporal Outcome by Treatment Status')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.show()
        
        # Now we can apply both spatial and temporal methods
        # Step 1: Create spatial features for each time period
        spatial_time_dfs = []
        
        # Process each time period separately for spatial dependencies
        for time in range(spatial_panel_df['time'].max() + 1):
            # Extract data for this time period
            time_data = spatial_panel_df[spatial_panel_df['time'] == time]
            
            # Create a temporary GeoDataFrame for this time period
            time_gdf = spatial_df.loc[time_data['unit_id']].copy()
            time_gdf['outcome'] = time_data['outcome'].values
            time_gdf['treatment'] = time_data['treatment'].values
            
            # Handle spatial dependencies
            sdh = SpatialDependencyHandler(weight_type='queen')
            sdh.fit(time_gdf)
            time_gdf_with_lags = sdh.transform(time_gdf)
            
            # Add time identifier
            time_gdf_with_lags['time'] = time
            
            # Store processed data
            spatial_time_dfs.append(time_gdf_with_lags)
        
        # Combine all time periods
        if len(spatial_time_dfs) > 0:
            combined_spatio_temporal = pd.concat(spatial_time_dfs)
            print(f"Created spatio-temporal dataset with {len(combined_spatio_temporal)} observations")
            
            # Step 2: Apply longitudinal method to spatial data
            spatio_temporal_handler = LongitudinalDataHandler(method='did')
            
            # Fit the handler
            spatio_temporal_handler.fit(
                X=combined_spatio_temporal,
                D=combined_spatio_temporal['treatment'],
                Y=combined_spatio_temporal['outcome'],
                time_var='time',
                id_var=combined_spatio_temporal.index
            )
            
            # Get the treatment effect estimate
            spatio_temporal_effect = spatio_temporal_handler.estimate_effect()
            print(f"\nSpatiotemporal treatment effect estimate: {spatio_temporal_effect:.3f}")
    except Exception as e:
        print(f"Error in spatio-temporal analysis: {e}")
else:
    print("Skipping spatio-temporal analysis due to missing dependencies.")

## 11. Conclusion

In this notebook, we've demonstrated how to handle spatial dependencies and longitudinal data in causal inference for active transportation research. Key takeaways:

1. **Spatial dependencies** can significantly impact effect estimates, and ignoring them may lead to biased results
2. **Longitudinal methods** like difference-in-differences and fixed effects help account for time-invariant confounders
3. **Event study analysis** provides insights into the temporal dynamics of treatment effects
4. **Combined spatio-temporal analysis** offers the most comprehensive approach for transportation interventions

These methods are particularly important for active transportation research because:
- Infrastructure improvements have spatial spillover effects to nearby areas
- Benefits of active transportation infrastructure often grow over time as behavior adapts
- Complex confounding factors like neighborhood characteristics require advanced methods